"""
The MIT License (MIT)

Copyright (c) 2017 Chris Scott

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM,
DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR
OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE
OR OTHER DEALINGS IN THE SOFTWARE.
"""

<a target="_blank" href="https://colab.research.google.com/github/ABMI-Kevin/Recognizer_CommonNighthawk/blob/main/CommonNighthawk_Recognizer_Processing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Adjust the Colab link!

# Import Packages

Import packages to run the model.  Updates to packages since the model was originally created may result in warnings when running the model.  As of March 2024 the most up to date versions of the packages below allowed the model to be run successfully on audio data despite any warnings that python may generate during the output.

In [1]:
from datetime import timedelta
import librosa
import numpy as np
import os
import pandas as pd
from scipy.special import expit
import tensorflow as tf
import time
from tqdm.notebook import tqdm_notebook

# Google Colab Steps

If running the notebook in Colab then you'll need to mount Google Drive and clone the repository to it in order to call the models and your recording files when processing.  Skip the next two steps if running on a local machine.


### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

### Clone the Github repository

If running the notebook through Colab then clone the Github repository to your Google Drive in order to access the model files stored in the repository.

In [ ]:
!git clone https://github.com/ABbiodiversity/Recognizer_Ovenbird.git

#Adjust this link

# Change your working directory
Change your working directory to the folder containing the notebook and the model files.

## If running locally

This will be the cloned repository or the folder where you downloaded the files if you didn't clone them with git.
```python
%cd "C:/Users/User/Folder/"
```

## If running in Colab

The location should be the location that the above step cloned into.  The output of the above step should look something like this:
```
Cloning into 'Recognizer_CommonNighthawk'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 44 (delta 19), reused 27 (delta 9), pack-reused 0
Receiving objects: 100% (44/44), 2.03 MiB | 8.96 MiB/s, done.
Resolving deltas: 100% (19/19), done.

```
Indicating, in this instance, that the github repository was cloned into 'Recognizer_CommonNighthawk'.  Therefore you will change the working directory to match that location:
```python
%cd Recognizer_CommonNighthawk
```

This cell will output the working directory so you can double check that it is pointing to the right directory.

In [2]:
# Change the working directory using the %cd command
%cd "C:/Users/Kevin Kelly/Documents/Repos/Recognizer_CommonNighthawk"

C:\Users\Kevin Kelly\Documents\Repos\Recognizer_CommonNighthawk


C:\Users\Kevin Kelly\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Designate Recording Folder

Enter the full filepath of the folder where the recording files to be processed are stored. The filepath will need to be changed to wherever you have stored the files on your machine or on Google Drive, depending on your processing choice.

Example filepath for local files: 
```python
"C:/Users/User/Documents/Ovenbird_recordings/"
```
If you're unfamiliar with the syntax for Google Drive paths in Colab, "/content/drive/" is required, followed by the rest of the filepath in your Google Drive folder structure, excluding any spaces in the folder names. Example filepath for Google Drive folder: 
```python
"/content/drive/MyDrive/Ovenbird/Recordings_Ovenbird/'
```

In [3]:
# Change path below to match your recordings folder filepath
RECORDINGS = 'C:/Users/Kevin Kelly/Documents/Repos/Recognizer_CommonNighthawk/Recordings/'

### Check the recording files
If you want to check that you have correctly designated the folderpath of your target recordings and that the folder contains the recording files then you can run the following cell to list all the contents of your RECORDINGS folder.

In [ ]:
os.listdir(RECORDINGS)

# Run the Common Nighthawk recognizer on your recordings

The following script will run the audio files in your "RECORDINGS" folder through the Common Nighthawk recognizer and export the results for each recording file.

In [4]:
def spectrogram_from_audio(y, sr):
    sr = 16000
    n_fft = 64
    hop_length = 32
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=np.blackman)[1:,:] # remove DC
    D = librosa.power_to_db(np.abs(D)**2, ref=1.0)
    D = np.swapaxes(D,0,1)[np.newaxis,np.newaxis,:,:] # [batch,height,width,channels]
    return D

def process_directory(recordings):
    # make sure there is somewhere to save results
    results_recognizer_location = os.path.join(recordings, 'results_recognizer')
    if not os.path.exists(results_recognizer_location):
        os.makedirs(results_recognizer_location)
    
    # find tensorflow model files
    meta_graph_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'coni-model.meta')
    checkpoint_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'coni-model')
    
    # list all audio files in directory
    audio_files = librosa.util.find_files(recordings, ext=['flac', 'wav'], recurse=True, case_sensitive=False, limit=None, offset=0)
    audio_files = list(set(audio_files)) # remove duplicates - not sure why find_files produces them!
    
    tf.compat.v1.reset_default_graph()
    with tf.compat.v1.Session() as sess:
        new_saver = tf.compat.v1.train.import_meta_graph(meta_graph_path)
        new_saver.restore(sess, checkpoint_path)
        features = tf.compat.v1.get_collection('input')[0]
        is_training = tf.compat.v1.get_collection('is_training')[0]
        level = tf.compat.v1.get_collection('level')[0]
        detection_fn = tf.compat.v1.get_collection('detection_fn')[0]
        
        for filename in tqdm_notebook(audio_files):
            y, sr = librosa.load(filename, sr=16000, mono=True)
            S = spectrogram_from_audio(y, sr)
            level_dB, detection_function = sess.run([level, detection_fn], feed_dict={features: S, is_training: False})
            level_dB = level_dB[0,0,:,0]
            detection_function = detection_function[0,0,:,0]
            duration_secs = y.size / float(sr) 
            times = np.interp(np.arange(detection_function.size), [0,detection_function.size-1], [0,duration_secs])
            df = pd.DataFrame({'detection_window_start_time': times, 'detector_score': detection_function, 'relative_sound_level': level_dB})
        
            base = os.path.basename(filename)
            filename = os.path.splitext(base)[0]
            savename = os.path.join(results_recognizer_location, filename + '_recognizer_function.csv')
            df.to_csv(savename, header=True, index=False)

process_directory(RECORDINGS)



'local_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\Kevin Kelly\Documents\Repos\Recognizer_CommonNighthawk\coni-model


C:\Users\Kevin Kelly\AppData\Local\Temp\ipykernel_29056\2300897843.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for filename in tqdm_notebook(audio_files):


  0%|          | 0/840 [00:00<?, ?it/s]

## Common Nighthawk Recognizer Results

Results files will be exported in csv format to a folder called "results_recognizer" within the RECORDINGS folder to keep the results and the recordings together.  The files will have the same name as the audio recordings files with "_detection_function" as a suffix.  Each row in the files will consist of the detection window start time, the recognizer score (between 0 and 1) for that detection window, and the relative sound level for that detection window.

---
---

# Distance Estimation

In addition to predictions about the presence of Common Nighthawks, predictions about the distance of the bird from the ARU can be made based on the relative sound level of the signal in the recording.  Yip et al. 2019 used calibrated datasets and developed a generalized linear model that can be used on other recordings with similar field and recording conditions.  The calibrated dataset used SM4 ARUs (WildLife Acoustics, Concord, MA, USA) in an open habitat, so the distance estimations may not be accurate if using an ARU with dramatically different recording settings, or if recordings are done in a closed habitat type since the relative sound levels in the recordings can be affected by density of vegetation around the ARU and ARU recording/microphone settings.  Additional details on the field methods and statistical modelling methods can be found in Yip et al. 2019.

This process that follows will produce distance estimates for Common Nighthawks in recordings that were run through the recognizer above.  It includes 95% confidence intervals, as displayed in the distance_CI975 and distance_CI025 fields.

## Setting up the data for the Distance Estimation model
If you want to estimate the distance of Common Nighthawks in the dataset that you have run through the recognizer above then you can use the following steps to compare the results of the detector to the distance estimation model developed in Yip et al. 2019.

Designate the folder that contains the results files from the recognizer step you want to run the distance estimator on.

In [47]:
#As entered here this should be the same folder location as the output of the recognizer step above, but if you have other CONI recognizer results that match the format of the output from this workbook then you can designate that folder as the results_location instead.
results_recognizer_location = os.path.join(RECORDINGS, 'results_recognizer')

Create a list of the csv files in the results_recognizer_location folder

In [48]:
results_recognizer_files = [os.path.join(results_recognizer_location, f) for f in os.listdir(results_recognizer_location) if f.endswith('.csv')]

Read in these csv files to separate dataframes and add some information to the dataframes.

In [50]:
individual_dataframes = []
for file in results_recognizer_files:
    df = pd.read_csv(file)
    df['filename'] = os.path.basename(file)  # Add a new column with the recognizer result file name
    df = df.map(lambda x: x.replace('_detection_function.csv', '') if isinstance(x, str) else x) # Remove filename additions added in recognizer step
    df['species'] = 'CONI' #Add species to dataframe
    individual_dataframes.append(df)

Finally combine each the individual recognizer results file's dataframes into a single dataframe.

In [51]:
distance_input = pd.concat(individual_dataframes, ignore_index=True)

## Generate Distance Predictions
This step takes the relative sound level from the classifier results and uses the generalized linear mixed model developed in Yip et al. 2019 to get an estimate of the distance of the Common Nighthawk calls in the dataset.

Import the Distance Predictions file, which is the result of the generalized linear model in 2019.

In [52]:
predictions_model = pd.read_csv("DistancePredictions.csv")

Adjust the number of decimal places in the relative sound level column to match the level column in the predictions data.

In [53]:
distance_input['level'] = distance_input['relative_sound_level'].round(1)

Match the sound level in the results dataframe to the predictions file to generate the distance estimates for each detection window.

In [54]:
distance_output = distance_input.merge(predictions_model, on=['level', 'species'], how='left')
distance_output = distance_output[['filename', 'detection_window_start_time', 'species', 'detector_score', 'relative_sound_level', 'distance', 'distance_CI975', 'distance_CI025']]

## Export distance estimate results

You can export the results of the distance estimation as one large csv of all results that were fed in, or split out as individual csv files, one for each file that was run through the distance estimation process.

Since you may be generating distance estimation results on multiple recording files at the same time you can either export the results as one large csv file with the name of the classifier results files (if you used the recognizer steps above these should be the same as the original recording filename) as a field in the csv, or you can export individual csv files that are named to match the name of the classifier results files that were imported to the function (again, if you used the classifier steps above these should retain the name of the original recording file).

Create the folder to store the distance estimation results.

In [60]:
results_distance_location = os.path.join(RECORDINGS, 'results_distance')
os.makedirs(results_distance_location, exist_ok=True)

### Single file output
Produces a single, combined results csv with the name of the classifer results files as a field within the csv.

In [61]:
distance_output.to_csv(os.path.join(results_distance_location, 'combined_distance_function.csv'), index=False)

### Multiple file output

Produces individual results files named for each results csv.

In [56]:
split_field = 'filename'
unique_values = distance_output[split_field].unique()

for value in unique_values:
    split_df = distance_output[distance_output[split_field] == value].drop(columns=[split_field])
    output_file = os.path.join(results_distance_location, f'{value}'+ '_distance_function.csv')
    split_df.to_csv(output_file, index=False)

# Note on Distance Estimation Interpretation

The results of the distance estimation process are dependent on the relative sound levels calculated by the recognizer but they do not consider whether the detection window in question actually contains a Common Nighthawk.  The distance estimator will estimate a distance for all detection windows regardless of the recognizer score, so users of the distance estimation values should make sure that they only use distance estimates from detection windows that have classifier scores above their calculated threshold for positive identification.

In addition, several consecutive detection windows may have a high recognizer scores, likely indicating that the Common Nighthawk call overlapped with several detection windows.  The distance estimation function will probably vary between these detection windows because the relative sound level in these detections windows will vary, so you can take the lowest (closest) distance estimation value from these consecutive detection windows to determine how close the Common Nighthawk was.

# References:
Yip, D. A., Knight, E. C., Haave‐Audet, E., Wilson, S. J., Charchuk, C., Scott, C. D., Solymos, P. & Bayne, E. M. (2020). Sound level measurements from audio recordings provide objective distance estimates for distance sampling wildlife populations. Remote Sensing in Ecology and Conservation, 6(3), 301-315.